<a href="https://colab.research.google.com/github/Ken-Shelley84/testing/blob/master/Data_Privacy_Workshop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Privacy Workshop

You recently received a file containing anonymized donor records for a charity in Ohio.  The records removed the name and address of the donors, but still included date of birth, gender, and ZIP code.

The records indicate that five donors, in particular, were invited to an exclusive (and fictitious) reception.  Here are the partial records for those donors:

| Birth Date | Gender | ZIP Code |
| --- | --- | --- |
| 12/30/1984 | M | 44333 |
| 5/13/1952 | M | 43082 |
| 7/10/1964 | M | 43017 |
| 10/3/1951 | F | 43235 |
| 7/23/1955 | F | 44106 |




Discover the identity of these five individuals and determine how each of these individuals voted in the past three Presidential elections using publicly available data


### Public Data: Ohio Voter Records
Ohio makes its voter records file available to the public.  Re-identify the five donors using the Ohio public voter records.


You may learn more about the contents of the statewide voter file in the [Voter File Layout](https://s3.amazonaws.com/ohio-voters/voter_file_layout.doc) Documentation.

The contents of the entire voter file are approximately 4GB in size.  They are available to you in four segments.  This data has already been loaded into a database for you.

*   https://s3.amazonaws.com/ohio-voters/SWVF_1_22.txt
*   https://s3.amazonaws.com/ohio-voters/SWVF_23_44.txt
*   https://s3.amazonaws.com/ohio-voters/SWVF_45_66.txt
*   https://s3.amazonaws.com/ohio-voters/SWVF_67_88.txt

Sidebar: If you are curious what information your state makes available, check it out [here](https://www.ncsl.org/research/elections-and-campaigns/access-to-and-use-of-voter-registration-lists.aspx)

### Public Data: Baby Names
The US Government makes name and gender data avilable in their baby names dataset.  The data (name, year of birth, sex, and number) are from a 100 percent sample of Social Security card applications for 1880 onward.

https://catalog.data.gov/dataset/baby-names-from-social-security-card-applications-national-level-data

This data has already been processed and the most likely gender for a name as been uploaded into a database for you.

### Snowflake Database
Our snowflake instance has two tables:


*   ohio_voter_records
*   name_gender



#### Install Snowflake Connector

In [0]:
# install the snowflake driver
!pip install --upgrade snowflake-connector-python[pandas] --quiet

### For google colab users only
Google co-lab's default version of SSL sockets is not compatible with Snowflake.  However, now that you've installed Snowflake.connector if you restart your runtime and you will have the correct version of ssl sockets.

**In your menu, click:**
Runtime -> Restart Runtime


### Load the necessary libraries

In [0]:
# load the necessary libraries
import snowflake.connector
import pandas as pd
import numpy as np

#### Setup Required Snowflake Database Paramaters

In [0]:
user='cfreder2' #@param {type:"string"}
password="NotreDame2020" #@param {type:"string"}
account='tua25689'
warehouse='COMPUTE_WH' 
database='DATA_PRIVACY'
schema='PUBLIC'

### Open the connection

In [11]:
# Connect to Snowflake
con = snowflake.connector.connect(
  user=user,
  password=password,
  account=account,
  warehouse=warehouse,
  database=database,
  schema=schema,
  autocommit=False)

# print our session id to prove we connected
print(con.database, con.session_id)

DATA_PRIVACY 19310780645


### Practice Querying the Database

Basic select syntax:

select [column names] from [table] where [predicate];

In [33]:
# top 10 limits the number of rows returned to 10.  The "*" simple means select all columns.
sql = "select top 10 * from OHIO_VOTER_RECORDS;"
cur = con.cursor()
cur.execute(sql)

In [0]:
# get the number of rows
cur.rowcount

In [35]:
# get the column description
cur.description

[('SOS_VOTERID', 2, None, 16777216, None, None, True),
 ('COUNTY_NUMBER', 2, None, 16777216, None, None, True),
 ('COUNTY_ID', 2, None, 16777216, None, None, True),
 ('LAST_NAME', 2, None, 16777216, None, None, True),
 ('FIRST_NAME', 2, None, 16777216, None, None, True),
 ('MIDDLE_NAME', 2, None, 16777216, None, None, True),
 ('SUFFIX', 2, None, 16777216, None, None, True),
 ('DATE_OF_BIRTH', 2, None, 16777216, None, None, True),
 ('REGISTRATION_DATE', 2, None, 16777216, None, None, True),
 ('VOTER_STATUS', 2, None, 16777216, None, None, True),
 ('PARTY_AFFILIATION', 2, None, 16777216, None, None, True),
 ('RESIDENTIAL_ADDRESS1', 2, None, 16777216, None, None, True),
 ('RESIDENTIAL_SECONDARY_ADDR', 2, None, 16777216, None, None, True),
 ('RESIDENTIAL_CITY', 2, None, 16777216, None, None, True),
 ('RESIDENTIAL_STATE', 2, None, 16777216, None, None, True),
 ('RESIDENTIAL_ZIP', 2, None, 16777216, None, None, True),
 ('RESIDENTIAL_ZIP_PLUS4', 2, None, 16777216, None, None, True),
 ('RESIDE

In [36]:
# fetch all the rows
results = con.cursor().execute(sql).fetchall()
for row in results:
  # Each row is a tuple
  print(row)

('OH0018137886', '23', '115412', 'POLT', 'TRICIA', 'L', '', '1974-06-11', '2014-10-05', 'ACTIVE', '', '317 W HIGH ST', '', 'AMANDA', 'OH', '43102', '', '', '', 'PO BOX 297', '', 'AMANDA', 'OH', '43102', '', '', '', 'EASTLAND-FAIRFIELD CAREER/TECH', '', '', '', '15', '05', 'FAIRFIELD COUNTY ESC', '', '', 'AMANDA-CLEARCREEK LOCAL SD (FAIRFIELD)', '', 'AMANDA B', '23-P-ABM', '09', '78', '20', 'TOWNSHIP OF AMANDA', 'AMANDA VILLAGE', '', 'D', '', '', '', '', 'X', '', 'X', 'X', 'X', '', '', '', 'X', '', '', 'X', '', '', '', '', '', 'D', '', 'X', '', 'X', '', '', '', 'X', 'D', '', '', 'X', 'X', '', 'X', 'R', 'X', '', '', '', '', '', 'X', '', '', '', '', '', '', '', '', '')
('OH0021956919', '44', '99427', 'DICK', 'DAVID', 'WAYNE', '', '1977-11-18', '2012-09-11', 'ACTIVE', '', '395 P R 5028 CO RD 2', '', 'CHESAPEAKE', 'OH', '45619', '', '', '', '395 PRIVATE ROAD 5028', '', 'CHESAPEAKE', 'OH', '45619', '', '', '', 'LAWRENCE COUNTY JVSD', '', '', '', '06', '04', 'LAWRENCE COUNTY ESC', '', '', 'SY

In [37]:
# fetch all the rows into a pandas dataframe
df = con.cursor().execute(sql).fetch_pandas_all()
df

,SOS_VOTERID,COUNTY_NUMBER,COUNTY_ID,LAST_NAME,FIRST_NAME,MIDDLE_NAME,SUFFIX,DATE_OF_BIRTH,REGISTRATION_DATE,VOTER_STATUS,PARTY_AFFILIATION,RESIDENTIAL_ADDRESS1,RESIDENTIAL_SECONDARY_ADDR,RESIDENTIAL_CITY,RESIDENTIAL_STATE,RESIDENTIAL_ZIP,RESIDENTIAL_ZIP_PLUS4,RESIDENTIAL_COUNTRY,RESIDENTIAL_POSTALCODE,MAILING_ADDRESS1,MAILING_SECONDARY_ADDRESS,MAILING_CITY,MAILING_STATE,MAILING_ZIP,MAILING_ZIP_PLUS4,MAILING_COUNTRY,MAILING_POSTAL_CODE,CAREER_CENTER,CITY,CITY_SCHOOL_DISTRICT,COUNTY_COURT_DISTRICT,CONGRESSIONAL_DISTRICT,COURT_OF_APPEALS,EDU_SERVICE_CENTER_DISTRICT,EXEMPTED_VILL_SCHOOL_DISTRICT,LIBRARY,LOCAL_SCHOOL_DISTRICT,MUNICIPAL_COURT_DISTRICT,PRECINCT_NAME,PRECINCT_CODE,...,PRIMARY_05_02_2006,GENERAL_11_07_2006,PRIMARY_05_08_2007,PRIMARY_09_11_2007,GENERAL_11_06_2007,PRIMARY_11_06_2007,GENERAL_12_11_2007,PRIMARY_03_04_2008,PRIMARY_10_14_2008,GENERAL_11_04_2008,GENERAL_11_18_2008,PRIMARY_05_05_2009,PRIMARY_09_08_2009,PRIMARY_09_15_2009,PRIMARY_09_29_2009,GENERAL_11_03_2009,PRIMARY_05_04_2010,PRIMARY_07_13_2010,PRIMARY_09_07_2010,GENERAL_11_02_2010,PRIMARY_05_03_2011,PRIMARY_09_13_2011,GENERAL_11_08_2011,PRIMARY_03_06_2012,GENERAL_11_06_2012,PRIMARY_05_07_2013,PRIMARY_09_10_2013,PRIMARY_10_01_2013,GENERAL_11_05_2013,PRIMARY_05_06_2014,GENERAL_11_04_2014,PRIMARY_05_05_2015,PRIMARY_09_15_2015,GENERAL_11_03_2015,PRIMARY_03_15_2016,GENERAL_06_07_2016,PRIMARY_09_13_2016,GENERAL_11_08_2016,PRIMARY_05_02_2017,PRIMARY_09_12_2017
0,OH0018137886,23,115412,POLT,TRICIA,L,,1974-06-11,2014-10-05,ACTIVE,,317 W HIGH ST,,AMANDA,OH,43102,,,,PO BOX 297,,AMANDA,OH,43102,,,,EASTLAND-FAIRFIELD CAREER/TECH,,,,15,05,FAIRFIELD COUNTY ESC,,,AMANDA-CLEARCREEK LOCAL SD (FAIRFIELD),,AMANDA B,23-P-ABM,...,,X,,,,,,D,,X,,X,,,,X,D,,,X,X,,X,R,X,,,,,,X,,,,,,,,,
1,OH0021956919,44,99427,DICK,DAVID,WAYNE,,1977-11-18,2012-09-11,ACTIVE,,395 P R 5028 CO RD 2,,CHESAPEAKE,OH,45619,,,,395 PRIVATE ROAD 5028,,CHESAPEAKE,OH,45619,,,,LAWRENCE COUNTY JVSD,,,,06,04,LAWRENCE COUNTY ESC,,,SYMMES VALLEY LOCAL SD (LAWRENCE),LAWRENCE-CO,PRECINCT WINDSOR 2,44ADF,...,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
2,OH0010151081,29,22077,FISHER,SALLIE,L,,1947-09-06,1980-09-08,ACTIVE,R,3686 OLDE WILLOW DR,,DAYTON,OH,45431,,,,,,,,,,,,GREENE COUNTY VOCATIONAL SCH DIST,BEAVERCREEK CITY,BEAVERCREEK CITY SD,,10,02,,,,,FAIRBORN,BEAVERCREEK 003,29AAW,...,,X,,,X,,,D,,X,,,,,,X,,,,X,X,,X,R,X,X,,,X,,X,,,X,R,,,X,,
3,OH0010161989,29,31729,HOEL,TIMOTHY,D,,1955-11-30,1993-02-16,ACTIVE,,1880 JAMESTOWN GUNNERSVILLE RD,,JAMESTOWN,OH,45335,,,,,,,,,,,,GREENE COUNTY VOCATIONAL SCH DIST,,,,10,02,GREENE COUNTY ESC,,,GREENEVIEW LOCAL SD (GREENE),XENIA,SILVERCREEK 425,29AHV,...,,X,,,,,,,,X,,,,,,,,,,,,,,,X,,,,,,,,,,,,,X,,
4,OH0010217657,29,144490,COURTNEY,REBECCA,A,,1963-05-17,2004-01-31,ACTIVE,R,2293 ABBEY LN,,XENIA,OH,45385,,,,,,,,,,,,GREENE COUNTY VOCATIONAL SCH DIST,,BEAVERCREEK CITY SD,,10,02,,,,,FAIRBORN,BEAVERCREEK 093,29AGT,...,D,X,,,X,,,D,,X,,,,,,X,,,,X,,,,,X,,,,X,,X,,,X,R,,,X,,
5,OH0011843600,41,5151730,KUGLER,TIFFINIE,MARIE,,1981-12-16,2006-12-22,ACTIVE,,171 LOCUST ST,,WINTERSVILLE,OH,43953,,,,,,,,,,,,JEFFERSON COUNTY JVSD,,,COUNTY COURT,06,07,JEFFERSON COUNTY ESC,,,INDIAN CREEK LOCAL SD (JEFFERSON),,PRECINCT WINTERSVILLE D,41ACI,...,,,,,,,,D,,X,,,,,,,,,,,,,,,,,,,,,,,,,,,,X,,
6,OH0011851509,41,3378200,ORLANDO,NORMA,JEAN,,1947-01-23,1972-10-10,ACTIVE,R,4526 FAIRWAY DR,,STEUBENVILLE,OH,43953,,,,,,,,,,,,JEFFERSON COUNTY JVSD,STEUBENVILLE CITY,,,06,07,JEFFERSON COUNTY ESC,,,INDIAN CREEK LOCAL SD (JEFFERSON),STEUBENVILLE,PRECINCT STEUBENVILLE 6-3,41AAW,...,,,,,,,,D,,X,,,,,,X,,,,X,,,,D,X,D,,,,,,,,,R,,,X,,
7,OH0012343311,30,2949,PYLE,KAREN,M,,1942-04-16,1970-09-23,ACTIVE,D,624 N 9TH ST,,CAMBRIDGE,OH,43725,,,,,,,,,,,,,CAMBRIDGE CITY,CAMBRIDGE CITY SD,,06,05,,,,,,PRECINCT CAMBRIDGE 3-B,30AAN,...,,X,,,X,,,D,,X,,,,,,X,,,,X,,,X,D,X,,,,,,X,,,X,D,,,X,,
8,OH0012342866,30,15566,PICKENPAUGH,JANET,MARIE,,1953-12-30,1986-05-06,CONFIRMATION,,223 HIGH AVE,,BYESVILLE,OH,43723,,,,,,,,,,,,,,,,06,05,,,,ROLLING HILLS LOCAL SD (GU

In [0]:
# use a predicate (or where clause) to find all Green Party Affilates and only return first and last name
sql = "select first_name, last_name from ohio_voter_records where party_affiliation = 'G'"
df = con.cursor().execute(sql).fetch_pandas_all()
df

In [0]:
# select one or more names from the name_gender table
sql = "select * from name_gender where first_name in ('CHRIS','CHRISTINA')"
df = con.cursor().execute(sql).fetch_pandas_all()
df

In [43]:
# return all voter records that are potentially females.
sql = """SELECT v.first_name,
                v.last_name,
                v.date_of_birth, 
                v.party_affiliation, 
                n.gender,
                v.residential_zip,
                v.GENERAL_11_04_2008,
                v.GENERAL_11_06_2012,
                v.GENERAL_11_08_2016
          FROM   ohio_voter_records AS v 
                INNER JOIN name_gender AS n 
                        ON v.first_name = n.first_name 
          WHERE  (v.date_of_birth = '1984-12-30' AND v.residential_zip = 	44333) OR 
                 (v.date_of_birth = '1952-05-13' AND v.residential_zip = 	43082) OR
                 (v.date_of_birth = '1964-07-10' AND v.residential_zip = 	43017 AND n.gender = 'M') OR
                 (v.date_of_birth = '1951-10-03' AND v.residential_zip = 	43235 AND n.gender = 'F' AND v.first_name != 'ESTHER') OR
                 (v.date_of_birth = '1955-07-23' AND v.residential_zip = 	44106 AND n.gender = 'F')
                 
      """
df = con.cursor().execute(sql).fetch_pandas_all()
df

,FIRST_NAME,LAST_NAME,DATE_OF_BIRTH,PARTY_AFFILIATION,GENDER,RESIDENTIAL_ZIP,GENERAL_11_04_2008,GENERAL_11_06_2012,GENERAL_11_08_2016
0,BARBARA,SNYDER,1955-07-23,D,F,44106,X,X,X
1,JOHN,KASICH,1952-05-13,R,M,43082,,X,X
2,KATHRYN,SULLIVAN,1951-10-03,,F,43235,X,X,X
3,LEBRON,JAMES,1984-12-30,,M,44333,X,,X
4,URBAN,MEYER,1964-07-10,,M,43017,,X,X


In [0]:
sql = """SELECT party_affiliation, 
              Count(*) AS number_female_voters 
         FROM ohio_voter_records AS v 
                INNER JOIN name_gender AS n 
                        ON v.first_name = n.first_name 
         WHERE  n.gender = 'F' 
         GROUP  BY party_affiliation 
         ORDER  BY number_female_voters DESC"""
df = con.cursor().execute(sql).fetch_pandas_all()
df

### Reidentity the data
Discover the identity of the each of the five donors invited to the exclusive reception using the Ohio Votor Records.

| Birth Date | Gender | ZIP Code |
| --- | --- | --- |
| 12/30/1984 | M | 44333 |
| 5/13/1952 | M | 43082 |
| 7/10/1964 | M | 43017 |
| 10/3/1951 | F | 43235 |
| 7/23/1955 | F | 44106 |

In [32]:
# Your Code Here
sql = """SELECT v.first_name,
                v.last_name,
                v.date_of_birth, 
                v.party_affiliation, 
                n.gender,
                v.residential_zip
          FROM   ohio_voter_records AS v 
                INNER JOIN name_gender AS n 
                        ON v.first_name = n.first_name 
          WHERE  (v.date_of_birth = '1984-12-30' AND v.residential_zip = 	44333) OR 
                 (v.date_of_birth = '1952-05-13' AND v.residential_zip = 	43082) OR
                 (v.date_of_birth = '1964-07-10' AND v.residential_zip = 	43017 AND n.gender = 'M') OR
                 (v.date_of_birth = '1951-10-03' AND v.residential_zip = 	43235 AND n.gender = 'F') OR
                 (v.date_of_birth = '1955-07-23' AND v.residential_zip = 	44106 AND n.gender = 'F')
      """
df = con.cursor().execute(sql).fetch_pandas_all()
df

,FIRST_NAME,LAST_NAME,DATE_OF_BIRTH,PARTY_AFFILIATION,GENDER,RESIDENTIAL_ZIP
0,BARBARA,SNYDER,1955-07-23,D,F,44106
1,ESTHER,HALL,1951-10-03,,F,43235
2,JOHN,KASICH,1952-05-13,R,M,43082
3,KATHRYN,SULLIVAN,1951-10-03,,F,43235
4,LEBRON,JAMES,1984-12-30,,M,44333
5,URBAN,MEYER,1964-07-10,,M,43017


### Determine voting history
Generate a _tidy_ dataset that identifies if any of these 5 individuals voted in the past three preseidential elections.  What is their likely party affiliation?

In [53]:
# Your Code Here
sql = """SELECT v.first_name,
                v.last_name,
                v.date_of_birth, 
                v.party_affiliation, 
                n.gender,
                v.residential_zip,
                v.GENERAL_11_04_2008,
                v.GENERAL_11_06_2012,
                v.GENERAL_11_08_2016
          FROM   ohio_voter_records AS v 
                INNER JOIN name_gender AS n 
                        ON v.first_name = n.first_name 
          WHERE  (v.date_of_birth = '1984-12-30' AND v.residential_zip = 	44333) OR 
                 (v.date_of_birth = '1952-05-13' AND v.residential_zip = 	43082) OR
                 (v.date_of_birth = '1964-07-10' AND v.residential_zip = 	43017 AND n.gender = 'M') OR
                 (v.date_of_birth = '1951-10-03' AND v.residential_zip = 	43235 AND n.gender = 'F' AND v.first_name != 'ESTHER') OR
                 (v.date_of_birth = '1955-07-23' AND v.residential_zip = 	44106 AND n.gender = 'F')
                 
      """
df = con.cursor().execute(sql).fetch_pandas_all()
df

,FIRST_NAME,LAST_NAME,DATE_OF_BIRTH,PARTY_AFFILIATION,GENDER,RESIDENTIAL_ZIP,GENERAL_11_04_2008,GENERAL_11_06_2012,GENERAL_11_08_2016
0,BARBARA,SNYDER,1955-07-23,D,F,44106,X,X,X
1,JOHN,KASICH,1952-05-13,R,M,43082,,X,X
2,KATHRYN,SULLIVAN,1951-10-03,,F,43235,X,X,X
3,LEBRON,JAMES,1984-12-30,,M,44333,X,,X
4,URBAN,MEYER,1964-07-10,,M,43017,,X,X
